<a href="https://colab.research.google.com/github/marciamart/Classificacao-Ecommerce/blob/main/classifica%C3%A7%C3%A3o_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/fatihkgg/ecommerce-product-images-18k")

#USAR API TOKEN DO KAGGLE

Dataset em  /content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES

In [ ]:
data = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES'

# Parte 1 - Análise do DataSet

## Objetivo e informações esperadas

Objetivo:
* Apresentar informações do dataset.

Informações esperadas:
* Integridade dos Arquivos
  1. Verifique se todas as imagens listadas no arquivo de informações
realmente existem no diretório de imagens e vice-versa.
  2. Verifique se todas as imagens estão no mesmo formato, ex: JPEG,
PNG, etc.
* Consistência dos Metadados
  1. Verifique se há valores ausentes nos metadados e como esses casos
são tratados.
  2. Verifique valores inconsistentes, por exemplo: dimensões de imagens
fora do esperado.
* Qualidade das Imagens
  1. Identifique imagens corrompidas que não podem ser abertas ou
processadas.
* Distribuição das Classes
  1. Verifique a distribuição das classes para identificar possíveis
desequilíbrios que possam afetar a modelagem
* Duplicatas
  1. Identifique imagens duplicadas que possam enviesar os resultados.
  2. Verifique duplicatas no arquivo de informações.

Informações complementares:
* Elaborar uma apresentação para mostrar os resultados.
* Apresentar o dataset de forma detalhada.
* Verifique quais itens em Informações esperadas podem ser aplicados no
dataset.

## **Integridade dos Arquivos**
A função a seguir verifica se todas as imagens no diretório estão no mesmo formato (JPEG, JPG, PNG). Ela retorna um dicionário com os formatos das imagens como chaves e suas respectivas quantidades como valores.

Saída esperada: {'JPEG': 18175}

In [ ]:
import os
from PIL import Image

def verificar_formato_imagem(data):
    formatos = {}
    for raiz, dirs, arquivos in os.walk(data):
        for arquivo in arquivos:
            if arquivo.endswith(('jpeg', 'jpg', 'png')):
                caminho_arquivo = os.path.join(raiz, arquivo)
                with Image.open(caminho_arquivo) as img:
                    formato = img.format
                    if formato not in formatos:
                        formatos[formato] = 0
                    formatos[formato] += 1
            else:
                formatos['Desconhecido'] = formatos.get('Desconhecido', 0) + 1

    return formatos

formatos_imagens = verificar_formato_imagem(data)
print(formatos_imagens)

## **Consistência dos Metadados**

Nessa etapa, foi analisado a qualidade e a integridade das imagens, para assegurar que não haverá maiores problemas nas etapas seguintes.

Foram levantados os seguintes:


*   Dimensões das imagens
*   Quantidade de megapixes
*   Recorrência de cores




In [ ]:
!apt install libimage-exiftool-perl
#Ferramenta utilizada para leitura de metadados de imagens

!pip install colorgram.py
# Lib utilizada para a extrair cores predominantes de uma imagem

#### **Dimensão das imagens**

Utilizando o conjunto de imagens amostrais, presentes na pasta (train), não foi encontrado alguma inconsistência, uma vez que o conjunto amostral apresenta imagens com as mesmas dimensões.

---
*Saída Esperada*:


```
A dimensão que mais aparece é: (224, 224) (quantidade: 551)
```






In [ ]:
import os
import subprocess
import imghdr
from collections import Counter

pasta_check = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check'

contagem_dimensoes = Counter()

def verificar_dimensoes_imagens(pasta):
    for item in os.listdir(pasta):
        caminho_item = os.path.join(pasta, item)

        if os.path.isdir(caminho_item):
            verificar_dimensoes_imagens(caminho_item)
        elif os.path.isfile(caminho_item) and imghdr.what(caminho_item) == 'jpeg':
            resultado = subprocess.run(["exiftool", "-s", "-ImageWidth", "-ImageHeight", caminho_item], capture_output=True, text=True)

            dimensoes = resultado.stdout.strip().split("\n")
            largura = int(dimensoes[0].split(": ")[1])
            altura = int(dimensoes[1].split(": ")[1])

            contagem_dimensoes[(largura, altura)] += 1

verificar_dimensoes_imagens(pasta_check)

dimensao_mais_comum = contagem_dimensoes.most_common(1)[0]

print(f"A dimensão que mais aparece é: {dimensao_mais_comum[0]} (quantidade: {dimensao_mais_comum[1]})")


#### **Quantidade de MegaPixels**

Também não foi encontrado dados ausentes ou inconsistentes nesse espaço de dados amostral, uma vez que todas as imagens apresentam a mesma quantidade de megapixels.

---
*Saída Esperada:*



```
Todos os arquivos possuem 0.050 megapixels.

```





In [ ]:
import os
import subprocess
import imghdr

pasta_check = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check'

megapixels_esperado = None
arquivos_com_megapixels_diferentes = []

def verificar_megapixels(pasta):
    global megapixels_esperado, arquivos_com_megapixels_diferentes

    for item in os.listdir(pasta):
        caminho_item = os.path.join(pasta, item)

        if os.path.isdir(caminho_item):
            verificar_megapixels(caminho_item)
        elif os.path.isfile(caminho_item) and imghdr.what(caminho_item) == 'jpeg':
            resultado = subprocess.run(["exiftool", "-s", "-Megapixels", caminho_item], capture_output=True, text=True)

            megapixels_atual = resultado.stdout.strip().split(": ")[1]

            if megapixels_esperado is None:
                megapixels_esperado = megapixels_atual
            elif megapixels_atual != megapixels_esperado:
                arquivos_com_megapixels_diferentes.append(caminho_item)

verificar_megapixels(pasta_check)

if arquivos_com_megapixels_diferentes:
    print("Nem todos os arquivos possuem o mesmo valor de megapixels.")
    print("Arquivos com megapixels diferentes:")
    for arquivo in arquivos_com_megapixels_diferentes:
        print(f"  - {arquivo}")
else:
    print(f"Todos os arquivos possuem {megapixels_esperado} megapixels.")


#### **Recorrência de Cores**

Nesse levantamento, foi observado que as cores semelhantes ao branco absoluto são as mais presentes nas categorias apresentadas. O algoritmo utilizado analisa a imagem como um todoo, por isso, o fundo "neutro" influênciou nesse resultado. Porém, utilizando o *OpenCV*, é possível detectar os contornos do produto e descobrir as cores além do fundo neutro.


---
*Saída utilizando apenas a biblioteca Colorgram:*



```
Categoria 'BABY_PRODUCTS': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 3.05
Categoria 'BEAUTY_HEALTH': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 2.03
Categoria 'CLOTHING_ACCESSORIES_JEWELLERY': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 6.29
Categoria 'ELECTRONICS': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 4.57
Categoria 'GROCERY': Cor predominante: Rgb(r=254, g=254, b=253), Proporção total: 9.50
Categoria 'HOBBY_ARTS_STATIONERY': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 2.84
Categoria 'HOME_KITCHEN_TOOLS': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 8.90
Categoria 'PET_SUPPLIES': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 2.11
Categoria 'SPORTS_OUTDOOR': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 12.45
```




In [ ]:
import os
import colorgram
from PIL import Image
import pandas as pd
import imghdr

pasta_check = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check'

def analisar_cores_por_categoria(pasta):
    dados = []

    for nome_subpasta in os.listdir(pasta):
        caminho_subpasta = os.path.join(pasta, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            categoria = nome_subpasta

            for item in os.listdir(caminho_subpasta):
                caminho_item = os.path.join(caminho_subpasta, item)
                if os.path.isfile(caminho_item) and imghdr.what(caminho_item) == 'jpeg':
                    try:
                        cores = colorgram.extract(Image.open(caminho_item), 5)  # Extrai 5 cores

                        for cor in cores:
                            dados.append({'categoria': categoria, 'cor_predominante': cor.rgb, 'proporcao': cor.proportion})
                    except Exception as e:
                        print(f"Erro ao processar a imagem {caminho_item}: {e}")

    df = pd.DataFrame(dados)

    cores_predominantes_por_categoria = df.groupby(['categoria', 'cor_predominante'])['proporcao'].sum().reset_index()
    cores_predominantes_por_categoria = cores_predominantes_por_categoria.loc[cores_predominantes_por_categoria.groupby('categoria')['proporcao'].idxmax()]

    for _, row in cores_predominantes_por_categoria.iterrows():
        print(f"Categoria '{row['categoria']}': Cor predominante: {row['cor_predominante']}, Proporção total: {row['proporcao']:.2f}")

analisar_cores_por_categoria(pasta_check)

Após utilizar a ferramenta *OpenCV*, conseguimos observar que, apesar da maioria das categorias ainda seguir na paleta de cores "branco" ou "cinza", a  categoria "BEAUTY_HEALTH" tem uma predominância de cores mais vibrantes e quentes, como o amarelo alaranjado, que podem transmitir a ideia de saúde, vitalidade e beleza, enquanto "BABY_PRODUCTS" tem uma predominância de cores claras e neutras, como o branco e o cinza claro, que são comumente associadas a produtos para bebês.

Cada categoria apresenta a seguinte variedade de cores:


*   BABY_PRODUCTS - **Cinza claro.**
*   HOME_KITCHEN_TOOLS - **Cinza bem claro.**
*   GROCERY - **Um branco com tons de amarelo.**
*   BEAUTY_HEALTH - **Dourado.**
*   CLOTHING_ACCESSORIES_JEWELLERY, ELECTRONICS, HOBBY_ARTS_STATIONERY, PET_SUPPLIES e SPORTS_OUTDOOR - **Branco.**


---

*Saída Esperada:*


>A proporção total indica o percentual que os metadados da categoria representam em relação ao total de dados.



```
Categoria 'BABY_PRODUCTS': Cor predominante: Rgb(r=237, g=237, b=236), Proporção total: 1.73
Categoria 'BEAUTY_HEALTH': Cor predominante: Rgb(r=226, g=165, b=4), Proporção total: 1.20
Categoria 'CLOTHING_ACCESSORIES_JEWELLERY': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 1.09
Categoria 'ELECTRONICS': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 3.55
Categoria 'GROCERY': Cor predominante: Rgb(r=254, g=254, b=253), Proporção total: 1.69
Categoria 'HOBBY_ARTS_STATIONERY': Cor predominante: Rgb(r=254, g=254, b=254), Proporção total: 1.20
Categoria 'HOME_KITCHEN_TOOLS': Cor predominante: Rgb(r=251, g=251, b=250), Proporção total: 1.61
Categoria 'PET_SUPPLIES': Cor predominante: Rgb(r=253, g=253, b=253), Proporção total: 1.76
Categoria 'SPORTS_OUTDOOR': Cor predominante: Rgb(r=252, g=252, b=252), Proporção total: 3.87

```







In [ ]:
import os
import colorgram
from PIL import Image
import pandas as pd
import imghdr
import cv2
import numpy as np

pasta_check = '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check'

def analisar_cores_por_categoria(pasta):
    dados = []

    for nome_subpasta in os.listdir(pasta):
        caminho_subpasta = os.path.join(pasta, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            categoria = nome_subpasta

            for item in os.listdir(caminho_subpasta):
                caminho_item = os.path.join(caminho_subpasta, item)
                if os.path.isfile(caminho_item) and imghdr.what(caminho_item) == 'jpeg':
                    try:
                        img = cv2.imread(caminho_item)
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                        _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

                        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        if len(contours) > 0:
                            maior_contorno = max(contours, key=cv2.contourArea)
                            x, y, w, h = cv2.boundingRect(maior_contorno)
                            img_recortada = img[y:y+h, x:x+w]

                            cores = colorgram.extract(Image.fromarray(img_recortada), 5)

                            for cor in cores:
                                dados.append({'categoria': categoria, 'cor_predominante': cor.rgb, 'proporcao': cor.proportion})
                        else:
                            print(f"Nenhum contorno encontrado na imagem {caminho_item}.")

                    except Exception as e:
                        print(f"Erro ao processar a imagem {caminho_item}: {e}")

    df = pd.DataFrame(dados)

    cores_predominantes_por_categoria = df.groupby(['categoria', 'cor_predominante'])['proporcao'].sum().reset_index()
    cores_predominantes_por_categoria = cores_predominantes_por_categoria.loc[cores_predominantes_por_categoria.groupby('categoria')['proporcao'].idxmax()]

    for _, row in cores_predominantes_por_categoria.iterrows():
        print(f"Categoria '{row['categoria']}': Cor predominante: {row['cor_predominante']}, Proporção total: {row['proporcao']:.2f}")

analisar_cores_por_categoria(pasta_check)


## **Qualidade de imagens**

In [ ]:
import os
import cv2
import numpy as np

#Verifica se uma imagem pode ser aberta.
def check_image(image_path):
  try:
    img = cv2.imread(image_path)
    return img is not None
  except Exception as e:
    print(f"Erro ao abrir {image_path}: {str(e)}")
    return False

#Encontra imagens corrompidas em um diretório e seus subdiretórios.
def find_corrupted_images(directory):
  """Encontra imagens corrompidas em um diretório e seus subdiretórios.

  Args:
    directory: Caminho para o diretório raiz.

  Returns:
    Uma lista com os caminhos das imagens corrompidas.
  """
  corrupted_images = []
  for root, _, files in os.walk(directory):
    for file in files:
      if file.endswith(('.jpg', '.jpeg', '.png')):  # Adapte para outros formatos se necessário
        image_path = os.path.join(root, file)
        if not check_image(image_path):
          corrupted_images.append(image_path)
  return corrupted_images

# Definindo os caminhos dos seus diretórios
train_dir = "/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/train"
val_dir = "/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/val"
test_dir = "/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check"

# Encontrando as imagens corrompidas em cada diretório
corrupted_train = find_corrupted_images(train_dir)
corrupted_val = find_corrupted_images(val_dir)
corrupted_test = find_corrupted_images(test_dir)

# Imprimindo ou salvando os resultados
print("Imagens corrompidas no conjunto de treinamento:")
print(corrupted_train)

##**Distribuição das Classes**
Conjuntos de treinamento (train) e validação (val) para treinamento de modelo e ajuste de hiperparâmetros. E uma pequena amostra, conjunto de verificação (check) para avaliação visual do modelo durante a implantação.

Aqui faremos uma análise da distribuição das imagens nessas classes.

####**TRAIN**

Aqui contaremos quantas imagens possui em cada classe (diretórios)

In [ ]:
import os
import pandas as pd

#análise da distribuição da classe: TRAIN
data =  '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/train'
subdiretorios = {}
# Passando por cada subdiretório
for dirpath, dirnames, filenames in os.walk(data):
    if dirpath == data: continue

    num_images = 0
    # Conta o número de imagens no subdiretório
    for filename in filenames:
        if filename.lower().endswith('.jpeg'):
            num_images += 1

    # Imprimi o número de imagens do subdiretório atual
    print(f"São {num_images} imagens no diretório {os.path.basename(dirpath)}")
    subdiretorios[os.path.basename(dirpath)] = num_images

print(f"Subdiretórios: {subdiretorios}")

Criando um gráfico de barras para visualização da distribuição das classes

In [ ]:
import matplotlib.pyplot as plt

plt.bar(subdiretorios.keys(), subdiretorios.values())
plt.xticks(rotation=90)
plt.ylabel("Quantidade de Imagens")
plt.title("Distribuição das Classes de Treinamento")
plt.show()

Com a análise podemos observar que a classe GROCERY possui significativamente mais instâncias em comparação com as demais classes, sugerindo um desequilíbrio considerável na distribuição das classes no conjunto de dados.

### **CHECK - VAL**
Faremos o mesmo para o conjunto de validação (val) e conjunto de verificação (check)

#### **VAL**

In [ ]:
import os
import pandas as pd

#análise da distribuição da classe: TRAIN
data =  '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/val'
subdiretorios = {}
# Passando por cada subdiretório
for dirpath, dirnames, filenames in os.walk(data):
    if dirpath == data: continue

    num_images = 0
    # Conta o número de imagens no subdiretório
    for filename in filenames:
        if filename.lower().endswith('.jpeg'):
            num_images += 1

    # Imprimi o número de imagens do subdiretório atual
    print(f"São {num_images} imagens no diretório {os.path.basename(dirpath)}")
    subdiretorios[os.path.basename(dirpath)] = num_images

print(f"Subdiretórios: {subdiretorios}")

In [ ]:
import matplotlib.pyplot as plt

plt.bar(subdiretorios.keys(), subdiretorios.values())
plt.xticks(rotation=90)
plt.ylabel("Quantidade de Imagens")
plt.title("Distribuição das Classes de Validação")
plt.show()

#### **CHECK**

In [ ]:
import os
import pandas as pd

#análise da distribuição da classe: TRAIN
data =  '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check'
subdiretorios = {}
# Passando por cada subdiretório
for dirpath, dirnames, filenames in os.walk(data):
    if dirpath == data: continue

    num_images = 0
    # Conta o número de imagens no subdiretório
    for filename in filenames:
        if filename.lower().endswith('.jpeg'):
            num_images += 1

    # Imprimi o número de imagens do subdiretório atual
    print(f"São {num_images} imagens no diretório {os.path.basename(dirpath)}")
    subdiretorios[os.path.basename(dirpath)] = num_images

print(f"Subdiretórios: {subdiretorios}")

In [ ]:
import matplotlib.pyplot as plt

plt.bar(subdiretorios.keys(), subdiretorios.values())
plt.xticks(rotation=90)
plt.ylabel("Quantidade de Imagens")
plt.title("Distribuição das Classes de Verificação")
plt.show()

## **Duplicatas**


1.   Identifique imagens duplicadas que possam enviesar os resultados.

2.   Verifique duplicatas no arquivo de informações.

In [ ]:
!pip install imagehash

In [ ]:
import os
from PIL import Image
import imagehash

# Diretórios contendo as imagens
data_dirs = [
    '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/check',
    '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/train',
    '/content/ecommerce-product-images-18k/ECOMMERCE_PRODUCT_IMAGES/val'
]

# Função para calcular hash de uma imagem
def calculate_image_hash(image_path):
    with Image.open(image_path) as img:
        return imagehash.phash(img)

# Identificar imagens duplicadas
image_hashes = {}
duplicates = []
dir_duplicate_counts = {dir: 0 for dir in data_dirs}

# Caminhar pelos diretórios de dados e processar as imagens
for data in data_dirs:
    for dirpath, dirnames, filenames in os.walk(data):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(dirpath, filename)
                img_hash = calculate_image_hash(image_path)

                if img_hash in image_hashes:
                    duplicates.append((filename, image_hashes[img_hash]))
                    # Incrementar o contador de duplicatas para o diretório atual
                    dir_duplicate_counts[data] += 1
                else:
                    image_hashes[img_hash] = filename

# Imprimir imagens duplicadas
print("Imagens duplicadas:")
for dup in duplicates:
    print(f"{dup[0]} -> {dup[1]}")

# Imprimir contagem de duplicatas por diretório
print("\nContagem de duplicatas por diretório:")
for dir, count in dir_duplicate_counts.items():
    print(f"{dir}: {count} duplicatas")

# Imprimir contagem total de duplicatas
total_duplicates = sum(dir_duplicate_counts.values())
print(f"\nTotal de duplicatas: {total_duplicates}")

# Se desejar salvar os resultados
# with open('duplicated_images.txt', 'w') as f:
#     for dup in duplicates:
#         f.write(f"{dup[0]} -> {dup[1]}\n")